In [128]:
import numpy as np
import glob
import pickle
import h5py
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [129]:
# for sake of simplicity we are not learning the rest time in between notes and chords

# load dataset

# get notes
def get_notes():
    notes = []
    
    i = 0
    for file in glob.glob("./data/tiersen/*.mid"):
        print(file)
#         print(i)
        i+=1
        midi = converter.parse(file)

        notes_to_parse = None
        parts = instrument.partitionByInstrument(midi)
        
        flag = False
        if parts: 
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in 
                                     element.normalOrder))
    with open('./data/tiersen/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    return notes

In [130]:
notes = get_notes()

In [131]:
# prepare_sequences
def prepare_sequences(notes, pitchnames, n_vocab):
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in
                      enumerate(pitchnames))

    network_in = []
    network_out = []

    #create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i+sequence_length]
        sequence_out = notes[i+sequence_length]
        network_in.append([note_to_int[char] for char in 
                                      sequence_in])
        network_out.append(note_to_int[sequence_out])

    n_patterns = len(network_in)

    #reshape the input into a format compatible with LSTM layers
    normalized_in = np.reshape(network_in, (n_patterns, 
                    sequence_length, 1))
    print(normalized_in.shape)

    # normalise input
    normalized_in = np.array(network_in) / float(n_vocab)
    normalized_in = np.reshape(normalized_in, (n_patterns,
                                sequence_length, 1))
    network_out = np_utils.to_categorical(network_out)
    
    return (network_in, normalized_in, network_out)

In [132]:
def create_net(network_in, n_vocab):
    model = Sequential()
    model.add(LSTM(256, 
        input_shape=(network_in.shape[1],network_in.shape[2]),
                  return_sequences=True
        ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', 
                 optimizer='rmsprop')
    return model

In [133]:
def train(model, network_in, network_out, epochs, batch_size):
    filepath = "optimalWeight_onlyBach.hdf5"    

    checkpoint = ModelCheckpoint(
        filepath, monitor='loss', 
        verbose=0,        
        save_best_only=True,        
        mode='min'
    )    
    callbacks_list = [checkpoint]     

    model.fit(network_in, network_out, epochs=epochs, batch_size=batch_size, callbacks=callbacks_list)

In [134]:
def pre_train_shit():
#     notes = get_notes()
    with open('./data/bach/notes', 'rb') as filepath:
        notes = pickle.load(filepath)
    print(len(notes))
    # get music vocabulary 
    n_vocab = len(set(notes))
    
    # get network inputs and outputs
    network_in, normalized_in, network_out = prepare_sequences(notes,  notes,
                                                n_vocab)
    # create network
    model = create_net(normalized_in, n_vocab)
    
    train(model, normalized_in, network_out, 200, 64)

## Prediction

In [147]:
def create_midi(output):
    offset = 0
    output_notes = []
    
    # create note and chord objects based on the values generated by the model
    for pattern in output:
        # if pattern in a chord
        if('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset=offset
            output_notes.append(new_chord)
        
        # if pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.storedInstrument = instrument.Piano()
            new_note.offset = offset
            output_notes.append(new_note)
            
        offset += 0.5
        
        midi = stream.Stream(output_notes)
        midi.write('midi', fp='tiersen2.mid')

In [148]:
def generate_notes(model, network_in, pitchnames, n_vocab):
#     start = np.random.randint(0, len(network_in)-1)
    start = 1
    
    
    int_to_note = dict((number, note) for number, note in
                      enumerate(pitchnames))
    
    pattern = network_in[start]
    output = []
    
    # generate 100 notes
    for note_index in range(100):
        prediction_input = np.reshape(pattern, 
                                      (1,len(pattern),1))
        prediction_input = prediction_input / float(n_vocab)
        
        prediction = model.predict(prediction_input, 
                                   verbose=0)
        
        index = np.random.choice(range(n_vocab), p=prediction.ravel())
        result = int_to_note[index]
        output.append(result)
        
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
    
    return output

In [149]:
def generate():
    with open('data/tiersen/notes', 'rb') as filepath:
        notes = pickle.load(filepath)
#     notes = notes[0:1000]

#     get all pitch names
    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(pitchnames)

    network_in, normalized_in, network_out = prepare_sequences(notes, pitchnames, n_vocab)
    
    
    model = create_net(normalized_in, n_vocab)
#     model.load_weights('./weights-improvement-182-0.0003-bigger.hdf5')
#     get output
    output = generate_notes(model, network_in, pitchnames,
                           n_vocab)
    create_midi(output)

In [138]:
pre_train_shit()

4586
(4486, 100, 1)
Epoch 1/200
4486/4486 [==============================] - 33s 7ms/step - loss: 3.8777
Epoch 2/200
4486/4486 [==============================] - 29s 7ms/step - loss: 3.6437
Epoch 3/200
4486/4486 [==============================] - 29s 7ms/step - loss: 3.6196
Epoch 4/200
4486/4486 [==============================] - 30s 7ms/step - loss: 3.6072
Epoch 5/200
4486/4486 [==============================] - 30s 7ms/step - loss: 3.6081
Epoch 6/200
4486/4486 [==============================] - 30s 7ms/step - loss: 3.5962
Epoch 7/200
4486/4486 [==============================] - 30s 7ms/step - loss: 3.5942
Epoch 8/200
4486/4486 [==============================] - 30s 7ms/step - loss: 3.5948
Epoch 9/200
4486/4486 [==============================] - 30s 7ms/step - loss: 3.5943
Epoch 10/200
4486/4486 [==============================] - 29s 7ms/step - loss: 3.5952
Epoch 11/200
4486/4486 [==============================] - 30s 7ms/step - loss: 3.5927
Epoch 12/200
4486/4486 [===================

4486/4486 [==============================] - 30s 7ms/step - loss: 3.5807
Epoch 97/200
4486/4486 [==============================] - 29s 7ms/step - loss: 3.5815
Epoch 98/200
4486/4486 [==============================] - 30s 7ms/step - loss: 3.5794
Epoch 99/200
4486/4486 [==============================] - 30s 7ms/step - loss: 3.5817
Epoch 100/200
4486/4486 [==============================] - 30s 7ms/step - loss: 4.2962
Epoch 101/200
4486/4486 [==============================] - 30s 7ms/step - loss: 3.5817
Epoch 102/200
4486/4486 [==============================] - 30s 7ms/step - loss: 3.5821
Epoch 103/200
4486/4486 [==============================] - 30s 7ms/step - loss: 3.5827
Epoch 104/200
4486/4486 [==============================] - 30s 7ms/step - loss: 3.5813
Epoch 105/200
4486/4486 [==============================] - 30s 7ms/step - loss: 3.5852
Epoch 106/200
4486/4486 [==============================] - 30s 7ms/step - loss: 3.5833
Epoch 107/200
4486/4486 [==============================] - 3

In [152]:
generate()

(3476, 100, 1)


In [120]:
from music21 import *

environment.set('musicxmlPath', '/usr/bin/musescore')
environment.set('midiPath', '/usr/bin/musescore')


In [11]:
littleMelody = converter.parse("test_outputFF.mid")
littleMelody.show('midi')

In [19]:
littleMelody2 = converter.parse("test_outputFF2.mid")
littleMelody2.show('midi')

In [21]:
# littleMelody2.show()

In [23]:
littleMelody3 = converter.parse("test_outputFF3.mid")
littleMelody3.show('midi')

In [24]:
littleMelody5 = converter.parse("test_outputFF5.mid")
littleMelody5.show('midi')

In [25]:
littleMelody6 = converter.parse("test_outputFF6.mid")
littleMelody6.show('midi')

In [121]:
littleMelody = converter.parse("tiersen1.mid")
littleMelody.show('midi')

In [153]:
littleMelody = converter.parse("tiersen2.mid")
littleMelody.show('midi')

In [151]:
littleMelody = converter.parse("bach1.mid")
littleMelody.show('midi')